In [2]:
!pip install langchain-text-splitters langchain-community langchain-ollama

  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.11
    Uninstalling langsmith-0.3.11:
      Successfully uninstalled langsmith-0.3.11
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40


In [4]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [13]:
!!pip install --quiet langchain_experimental langchain_openai

[]

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings
text = """
The Apollo 11 mission was a landmark achievement in human history. 
On July 20, 1969, astronauts Neil Armstrong and Buzz Aldrin became the first humans to walk on the moon. 
The mission's primary objective was to complete a national goal set by President John F. Kennedy. 
The Saturn V rocket, a technological marvel, propelled the astronauts on their journey.
Data collected during the mission provided invaluable insights into the moon's geology.

Deep-sea exploration presents its own set of unique challenges. 
Dr. Robert Ballard is famous for discovering the wreck of the Titanic in 1985.
He used a remotely operated vehicle (ROV) named Argo to survey the ocean floor.
The immense pressure and complete darkness of the deep ocean require specialized equipment.
Hydrothermal vents, discovered in the deep sea, host unique ecosystems that thrive without sunlight.
"""
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en")

In [19]:
from langchain_experimental.text_splitter import SemanticChunker
from rich import print
text_splitter = SemanticChunker(embedding)
text_splitter = SemanticChunker(
    embedding, breakpoint_threshold_type="percentile" # "standard_deviation", "interquartile"
)
documents = text_splitter.create_documents([text])
print(documents)

[
    Document(
        metadata={},
        page_content="\nThe Apollo 11 mission was a landmark achievement in human history. On July 20, 1969, 
astronauts Neil Armstrong and Buzz Aldrin became the first humans to walk on the moon. The mission's primary 
objective was to complete a national goal set by President John F. Kennedy. The Saturn V rocket, a technological 
marvel, propelled the astronauts on their journey. Data collected during the mission provided invaluable insights 
into the moon's geology. Deep-sea exploration presents its own set of unique challenges."
    ),
    Document(
        metadata={},
        page_content='Dr. Robert Ballard is famous for discovering the wreck of the Titanic in 1985. He used a 
remotely operated vehicle (ROV) named Argo to survey the ocean floor. The immense pressure and complete darkness of
the deep ocean require specialized equipment. Hydrothermal vents, discovered in the deep sea, host unique 
ecosystems that thrive without sunlight. '
    )
]

In [22]:
import os
from typing import List
# from dotenv import load_dotenv

from langchain import hub
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field

# --- Setup ---
# Load environment variables from .env file
# load_dotenv()

# Define the structured data model we want to extract
class Sentences(BaseModel):
    """A list of sentences."""
    sentences: List[str] = Field(description="A list of sentences extracted from the text.")

# --- LLM and Prompt Initialization ---

# Pull the pre-made prompt for propositionizing text from LangChain Hub
obj = hub.pull("wfh/proposal-indexing")

# Initialize the ChatGroq LLM
# The API key is read automatically from the GROQ_API_KEY environment variable
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key="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"
)

# --- Chains ---

# This is the modern way to create an extraction chain.
# It chains the LLM with the desired output structure (our Sentences class).
extractor = llm.with_structured_output(Sentences)

# This is the first step, which processes the raw text using the prompt from the Hub.
runnable = obj | llm

def get_propositions(text: str) -> List[str]:
    """
    Extracts a list of propositions from a given text.
    
    Args:
        text: The input text paragraph.
    
    Returns:
        A list of strings, where each string is a proposition.
    """
    # Step 1: Get the initial processed text from the hub prompt
    runnable_output = runnable.invoke({"input": text}).content
    
    # Step 2: Use the new structured output extractor to get the list of sentences
    # This replaces the old, problematic create_extraction_chain_pydantic
    structured_output = extractor.invoke(runnable_output)
    
    return structured_output.sentences

# --- Main Execution ---

# Sample text to process
text = """
The Apollo 11 mission was a landmark achievement in human history. On July 20, 1969, astronauts Neil Armstrong and Buzz Aldrin became the first humans to walk on the moon. The mission's primary objective was to complete a national goal set by President John F. Kennedy. The Saturn V rocket, a technological marvel, propelled the astronauts on their journey.

Deep-sea exploration presents its own set of unique challenges. Dr. Robert Ballard is famous for discovering the wreck of the Titanic in 1985. He used a remotely operated vehicle (ROV) named Argo to survey the ocean floor. The immense pressure and complete darkness of the deep ocean require specialized equipment.
"""

# Split the text into paragraphs and process the first 5
paragraphs = text.strip().split("\n\n")
text_propositions = []

print("Starting proposition extraction...")
for i, para in enumerate(paragraphs[:5]):
    if para: # Ensure the paragraph is not empty
        propositions = get_propositions(para)
        text_propositions.extend(propositions)
        print(f"Done processing paragraph {i+1}...")

print("\n" + "="*30)
print(f"Extraction complete. You have {len(text_propositions)} propositions.")
print("First 10 propositions:")
for i, prop in enumerate(text_propositions[:10]):
    print(f"{i+1}. {prop}")

/Users/abhijit/miniforge3/envs/chatbot/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Starting proposition extraction...

Done processing paragraph 1...

Done processing paragraph 2...

==============================

...

6. The Saturn V rocket was a technological marvel.

7. The Saturn V rocket propelled the astronauts on their journey.

8. The astronauts were Neil Armstrong and Buzz Aldrin.

9. Deep-sea exploration presents its own set of unique challenges.

10. Dr. Robert Ballard is famous.

In [23]:
import uuid
import os
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from pydantic import BaseModel  # <-- THIS IS THE CORRECTED LINE
from langchain_core.output_parsers import PydanticOutputParser
# from dotenv import load_dotenv
from rich import print

class AgenticChunker:
    def __init__(self, groq_api_key=None):
        """
        Initializes the AgenticChunker.
        It's recommended to set the GROQ_API_KEY in a .env file.
        """
        # load_dotenv() # Loads environment variables from a .env file
        
        self.chunks = {}
        self.id_truncate_limit = 5

        # Whether or not to update/refine summaries and titles as you get new information
        self.generate_new_metadata_ind = True
        self.print_logging = True

        # if groq_api_key is None:
        #     groq_api_key = os.getenv("GROQ_API_KEY")

        # if groq_api_key is None:
        #     raise ValueError("Groq API key is not provided. Please set it as an environment variable 'GROQ_API_KEY' in a .env file.")

        self.llm  = ChatGroq(
            temperature=0.2,
            model_name="llama3-70b-8192", # Recommended model for this complexity
            api_key="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"
        )

    def add_propositions(self, propositions):
        for proposition in propositions:
            self.add_proposition(proposition)
    
    def add_proposition(self, proposition):
        if self.print_logging:
            print (f"\n[bold]Adding[/bold]: '{proposition}'")

        # If it's your first chunk, just make a new chunk and don't check for others
        if len(self.chunks) == 0:
            if self.print_logging:
                print ("[italic]No chunks exist, creating a new one...[/italic]")
            self._create_new_chunk(proposition)
            return

        chunk_id = self._find_relevant_chunk(proposition)

        # If a chunk was found then add the proposition to it
        if chunk_id:
            if self.print_logging:
                print (f"[green]Chunk Found ({self.chunks[chunk_id]['chunk_id']}), adding to: {self.chunks[chunk_id]['title']}[/green]")
            self.add_proposition_to_chunk(chunk_id, proposition)
            return
        else:
            if self.print_logging:
                print ("[yellow]No relevant chunk found, creating a new one...[/yellow]")
            # If a chunk wasn't found, then create a new one
            self._create_new_chunk(proposition)
        
    def add_proposition_to_chunk(self, chunk_id, proposition):
        # Add proposition
        self.chunks[chunk_id]['propositions'].append(proposition)

        # Update summary and title
        if self.generate_new_metadata_ind:
            self.chunks[chunk_id]['summary'] = self._update_chunk_summary(self.chunks[chunk_id])
            self.chunks[chunk_id]['title'] = self._update_chunk_title(self.chunks[chunk_id])

    def _update_chunk_summary(self, chunk):
        """
        If you add a new proposition to a chunk, you may want to update the summary or else they could get stale
        """
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic.
                    A new proposition was just added to one of your chunks. Generate a very brief, 1-sentence summary for the chunk.
                    A good summary will say what the chunk is about and give clarifying instructions on what else to add.
                    Your summaries should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or a month, generalize it to "date and times".

                    Example:
                    Input: Proposition: Greg likes to eat pizza
                    Output: This chunk contains information about the types of food Greg likes to eat.

                    Only respond with the new chunk summary, nothing else.
                    """,
                ),
                ("user", "Chunk's propositions:\n{propositions}\n\nCurrent chunk summary:\n{current_summary}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_summary = runnable.invoke({
            "propositions": "\n".join(chunk['propositions']),
            "current_summary" : chunk['summary']
        }).content

        return new_chunk_summary
    
    def _update_chunk_title(self, chunk):
        """
        If you add a new proposition to a chunk, you may want to update the title or else it can get stale
        """
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic.
                    A new proposition was just added to one of your chunks. Generate a very brief, updated chunk title.
                    A good title will say what the chunk is about.
                    Your titles should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or a month, generalize it to "date and times".

                    Example:
                    Input: Summary: This chunk is about dates and times that the author talks about
                    Output: Date & Times

                    Only respond with the new chunk title, nothing else.
                    """,
                ),
                ("user", "Chunk's propositions:\n{propositions}\n\nChunk summary:\n{current_summary}\n\nCurrent chunk title:\n{current_title}"),
            ]
        )

        runnable = PROMPT | self.llm

        updated_chunk_title = runnable.invoke({
            "propositions": "\n".join(chunk['propositions']),
            "current_summary" : chunk['summary'],
            "current_title" : chunk['title']
        }).content

        return updated_chunk_title

    def _get_new_chunk_summary(self, proposition):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic.
                    Generate a very brief, 1-sentence summary for a new chunk based on the given proposition.
                    A good summary will say what the chunk is about and give clarifying instructions on what else to add.
                    Your summaries should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or a month, generalize it to "date and times".

                    Example:
                    Input: Proposition: Greg likes to eat pizza
                    Output: This chunk contains information about the types of food Greg likes to eat.

                    Only respond with the new chunk summary, nothing else.
                    """,
                ),
                ("user", "Determine the summary of the new chunk that this proposition will go into:\n{proposition}"),
            ]
        )

        runnable = PROMPT | self.llm
        return runnable.invoke({"proposition": proposition}).content
    
    def _get_new_chunk_title(self, summary):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic.
                    Generate a very brief, few-word chunk title based on the given chunk summary.
                    A good chunk title is brief but encompasses what the chunk is about.
                    Your titles should anticipate generalization.

                    Example:
                    Input: Summary: This chunk is about dates and times that the author talks about.
                    Output: Date & Times

                    Only respond with the new chunk title, nothing else.
                    """,
                ),
                ("user", "Determine the title of the chunk that this summary belongs to:\n{summary}"),
            ]
        )

        runnable = PROMPT | self.llm
        return runnable.invoke({"summary": summary}).content

    def _create_new_chunk(self, proposition):
        new_chunk_id = str(uuid.uuid4())[:self.id_truncate_limit]
        new_chunk_summary = self._get_new_chunk_summary(proposition)
        new_chunk_title = self._get_new_chunk_title(new_chunk_summary)

        self.chunks[new_chunk_id] = {
            'chunk_id' : new_chunk_id,
            'propositions': [proposition],
            'title' : new_chunk_title,
            'summary': new_chunk_summary,
            'chunk_index' : len(self.chunks)
        }
        if self.print_logging:
            print (f"[bold blue]Created new chunk ({new_chunk_id}): {new_chunk_title}[/bold blue]")
    
    def get_chunk_outline(self):
        """
        Get a string which represents the chunks you currently have.
        """
        if not self.chunks:
            return "No chunks yet."
            
        chunk_outline = ""
        for chunk_id, chunk in self.chunks.items():
            single_chunk_string = f"""Chunk ({chunk['chunk_id']}): {chunk['title']}\nSummary: {chunk['summary']}\n\n"""
            chunk_outline += single_chunk_string
        return chunk_outline

    def _find_relevant_chunk(self, proposition):
        current_chunk_outline = self.get_chunk_outline()

        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    Determine if the "Proposition" belongs to any existing chunks based on semantic similarity.
                    If it belongs, return ONLY the chunk_id.
                    If not, return ONLY the string "No chunks".
                    
                    Example 1:
                    Proposition: "Greg really likes hamburgers"
                    Current Chunks:
                    Chunk (2n4l3): Places in San Francisco
                    Summary: Things to do in San Francisco.
                    
                    Chunk (9383k): Food Greg Likes
                    Summary: Lists of food and dishes that Greg likes.
                    Output: 9383k

                    Example 2:
                    Proposition: "The sky is blue."
                    Current Chunks:
                    Chunk (9383k): Food Greg Likes
                    Summary: Lists of food and dishes that Greg likes.
                    Output: No chunks
                    """,
                ),
                ("user", "Current Chunks:\n---\n{current_chunk_outline}\n---\nProposition to evaluate: '{proposition}'"),
            ]
        )

        runnable = PROMPT | self.llm
        chunk_found_response = runnable.invoke({
            "proposition": proposition,
            "current_chunk_outline": current_chunk_outline
        }).content

        class ChunkID(BaseModel):
            """Extracting the chunk id"""
            chunk_id: Optional[str] = None

        parser = PydanticOutputParser(pydantic_object=ChunkID)

        extraction_prompt = ChatPromptTemplate.from_template(
            """
            Parse the user's text to find a chunk ID. The chunk ID is a short alphanumeric string. 
            If no chunk ID is found, return null for the chunk_id field.
            {format_instructions}
            Text to parse:
            {text_to_parse}
            """,
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )

        extraction_chain = extraction_prompt | self.llm | parser

        try:
            parsed_result = extraction_chain.invoke({"text_to_parse": chunk_found_response})
            chunk_found = parsed_result.chunk_id
        except Exception as e:
            if self.print_logging:
                print(f"[red]Could not parse chunk ID from response: '{chunk_found_response}'. Error: {e}[/red]")
            chunk_found = None

        if chunk_found is None or chunk_found not in self.chunks:
            return None

        return chunk_found
    
    def get_chunks(self, get_type='dict'):
        """
        Returns chunks as a dictionary or a list of joined strings.
        """
        if get_type == 'dict':
            return self.chunks
        if get_type == 'list_of_strings':
            return [" ".join(chunk['propositions']) for chunk in self.chunks.values()]
        return self.chunks
    
    def pretty_print_chunks(self):
        print(f"\n\n[bold underline]Final Chunks ({len(self.chunks)} total)[/bold underline]\n")
        for chunk_id, chunk in self.chunks.items():
            print(f"[bold]Chunk #{chunk['chunk_index']} - {chunk['title']}[/bold]")
            print(f"  [cyan]ID:[/cyan] {chunk_id}")
            print(f"  [cyan]Summary:[/cyan] {chunk['summary']}")
            print(f"  [cyan]Propositions ({len(chunk['propositions'])}):[/cyan]")
            for prop in chunk['propositions']:
                print(f"    - {prop}")
            print("\n")

    def pretty_print_chunk_outline(self):
        print("[bold underline]Final Chunk Outline[/bold underline]\n")
        print(self.get_chunk_outline())


if __name__ == "__main__":
    ac = AgenticChunker()

    # propositions = [
    #     'The month is October.',
    #     'The year is 2023.',
    #     "One of the most important things that I didn't understand about the world as a child was the degree to which the returns for performance are superlinear.",
    #     'Teachers and coaches implicitly told us that the returns were linear.',
    #     "I heard a thousand times that 'You get out what you put in.'",
    #     'Teachers and coaches meant well.',
    #     "The statement that 'You get out what you put in' is rarely true.",
    #     "If your product is only half as good as your competitor's product, you do not get half as many customers.",
    #     "You get no customers if your product is only half as good as your competitor's product.",
    #     'You go out of business if you get no customers.',
    #     'The returns for performance are superlinear in business.',
    #     'Some people think the superlinear returns for performance are a flaw of capitalism.',
    #     'Some people think that changing the rules of capitalism would stop the superlinear returns for performance from being true.',
    #     'Superlinear returns for performance are a feature of the world.',
    #     'Superlinear returns for performance are not an artifact of rules that humans have invented.',
    #     'The same pattern of superlinear returns is observed in fame.',
    #     'The same pattern of superlinear returns is observed in power.',
    #     'The same pattern of superlinear returns is observed in military victories.',
    #     'The same pattern of superlinear returns is observed in knowledge.',
    #     'The same pattern of superlinear returns is observed in benefit to humanity.',
    #     'In fame, power, military victories, knowledge, and benefit to humanity, the rich get richer.'
    # ]
    
    ac.add_propositions(text_propositions)
    ac.pretty_print_chunks()
    ac.pretty_print_chunk_outline()

Adding: 'The Apollo 11 mission was a landmark achievement in human history.'

No chunks exist, creating a new one...

Created new chunk (ba06f): Historical Events

Adding: 'The Apollo 11 mission occurred on July 20, 1969.'

Chunk Found (ba06f), adding to: Historical Events

...

Summary: This chunk contains information about the environmental conditions of different natural settings.

Propositions (1):

- The deep ocean has complete darkness.

Final Chunk Outline

Chunk (ba06f): Historical Events & Dates
Summary: This chunk contains information about significant events and their corresponding dates.

Chunk (3c14d): Exploration & Discovery
Summary: This chunk contains information about notable achievements, challenges, and key figures in exploration, 
including space and oceanic endeavors.

Chunk (5c2ce): Extreme Environment Technology
Summary: This chunk contains information about tools and technology used in exploring extreme environments.

Chunk (8bf96): Environmental Conditions
Summary: This chunk contains information about the environmental conditions of different natural settings.